In [1]:
var csv = require('csv-parser');  

In [2]:
var fs = require('fs');

In [6]:
var csv_train_file_path = './data/train.csv'

In [8]:
var X = []
var toxic_y = []
var severe_toxic_y = []

fs.createReadStream(csv_train_file_path)  
  .pipe(csv())
  .on('data', (row) => {
    X.push(row.comment_text);
    toxic_y.push(row.toxic)
    severe_toxic_y.push(row.severe_toxic)
  })
  .on('end', () => {
    console.log('CSV file successfully processed');
  });

CsvParser {
  _readableState:
   ReadableState {
     objectMode: true,
     highWaterMark: 16,
     buffer: BufferList { head: null, tail: null, length: 0 },
     length: 0,
     pipes: null,
     pipesCount: 0,
     flowing: true,
     ended: false,
     endEmitted: false,
     reading: false,
     sync: false,
     needReadable: false,
     emittedReadable: false,
     readableListening: false,
     resumeScheduled: true,
     paused: false,
     emitClose: true,
     autoDestroy: false,
     destroyed: false,
     defaultEncoding: 'utf8',
     awaitDrain: 0,
     readingMore: false,
     decoder: null,
     encoding: null },
  readable: true,
  _events:
   [Object: null prototype] {
     prefinish: [Function: prefinish],
     unpipe: [Function: onunpipe],
     drain: [Function: pipeOnDrainFunctionResult],
     error: [Function: onerror],
     close:
      { [Function: bound onceWrapper] listener: [Function: onclose] },
     finish:
      { [Function: bound onceWrapper] listener: [F

CSV file successfully processed


In [ ]:
var normalize = require('normalize');

In [9]:
doc = nlp("the guest-singer's björk   at seven thirty.").normalize().out('text')

ReferenceError: nlp is not defined